# Summary table

| Method                | Lower outlier threshold | Upper outlier threshold | pct of outliers |
|-----------------------|-------------------------|-------------------------|-----------------|
| Z-score 3 std         |                         |                         |                 |
| Box plot 1.5 IQR      |                         |                         |                 |
| Box plot 3 IQR        |                         |                         |                 |
| Cook's distance t=4/N | -                       | -                       |                 |



# Imports

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import seaborn as sns

#increase font size of all elements
sns.set(font_scale=1.5)


# Load the data

In [ ]:
import os
import requests

# Data can be found at https://www.kaggle.com/datasets/mustafaali96/weight-height and also at the following url
url = "https://gist.githubusercontent.com/nstokoe/7d4717e96c21b8ad04ec91f361b000cb/raw/bf95a2e30fceb9f2ae990eac8379fc7d844a0196/weight-height.csv"

def download_data(url, force_download=False, **read_csv_kwargs):
    # Donwload data if it is not in disk
    data_path = os.path.join('data', os.path.basename(url.split('?')[0]))
    if not os.path.exists(data_path) or force_download:
        # ensure data dir is created
        os.makedirs('data', exist_ok=True)
        # request data from url
        response = requests.get(url, allow_redirects=True)
        # save file
        with open(data_path, "w") as f:
            # Note the content of the response is in binary form: 
            # it needs to be decoded.
            # The response object also contains info about the encoding format
            # which we use as argument for the decoding
            f.write(response.content.decode(response.apparent_encoding))
    print(f"File saved to {data_path}")

download_data(url)


In [ ]:
# read csv file
data = pd.read_csv('data/weight-height.csv')
#Convert inches/pounds to m/kg
data.loc[:, 'Weight'] /= 2.205
data.loc[:, 'Height'] /= 39.37

# visualize the first 5 rows
data.head()


# Plot histograms


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, figsize=(16,4))
sns.histplot(data=data, x='Height', kde=True, ax=ax1)
sns.histplot(data=data, x='Weight', kde=True, ax=ax2)
fig.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,4))
sns.histplot(data=data, x='Height', hue='Gender', kde=True, ax=ax1)
sns.histplot(data=data, x='Weight', hue='Gender', kde=True, ax=ax2)

# Print basic statistics wtih `describe`


In [ ]:
display(data.describe())

# Z-score

In [ ]:
from scipy.stats import zscore
# help(zscore)

## Outlier ratio

In [ ]:
# Compute z-score
data_with_z = data.join(data[['Height', 'Weight']].apply(zscore), rsuffix='_z_score')

# compute outlier masks for Height and Weight
is_outlier_H = (data_with_z.Height_z_score.abs() > 3)
is_outlier_W = (data_with_z.Weight_z_score.abs() > 3)

# TODO complete to show the outlier count and ratio
# Display outlier count for height
outlier_count = ...
outlier_ratio = ...
print(
    f'Height outlier count: {outlier_count:d}/{data.shape[0]} '
    f'({outlier_ratio*100:0.5f}%)')
# Display the outlier count for width
outlier_count = ...
outlier_ratio = ...
print(
    f'Weight outlier count: {outlier_count:d}/{data.shape[0]} '
    f'({outlier_ratio*100:0.5f}%)')

**Note**: z-score can also be applied by group:

In [ ]:
data.join(data.groupby('Gender').apply(zscore), rsuffix='_z_score')

# Box-plots + interquartile range


## Whiskers at 1.5 IQR

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,5))
sns.boxplot(data=data, x='Height',orient='horiz', ax=ax1)
sns.boxplot(data=data, x='Weight',orient='horiz', ax=ax2)
fig.tight_layout()

## Whiskers at 3 IQR

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,5))
sns.boxplot(data=data, x='Height',orient='horiz', whis=3, ax=ax1)
sns.boxplot(data=data, x='Weight',orient='horiz', whis=3, ax=ax2)
fig.tight_layout()

## Outlier ratio

In [ ]:
q1 = data.quantile(0.25, interpolation='nearest')
q3 = data.quantile(0.75, interpolation='nearest')
IQR = q3-q1
IQR

In [ ]:
k = 1.5
lower, upper = q1-k*IQR, q3+k*IQR
lower, upper

In [ ]:
upper_outliers = data.Height > upper.loc['Height']
lower_outliers = data.Height < lower.loc['Height']
# TODO complete to retrieve outlier count and ratio
outilier_count = ...
outlier_ratio = ...
print(
    f'Height outlier count: {outilier_count:d}/{data.shape[0]} '
    f'({outlier_ratio*100:0.5f}%)')


In [ ]:
upper_outliers = data.Weight > upper.loc['Weight']
lower_outliers = data.Weight < lower.loc['Weight']
# TODO complete to retrieve outlier count and ratio
outilier_count = ...
outlier_ratio = ...
print(
    f'Weight outlier count: {outilier_count:d}/{data.shape[0]} '
    f'({outlier_ratio*100:0.5f}%)')


# Regression based methods

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
ax = axs[0]
ax.set_title('scatter+ regression line')
sns.regplot(
        data=data, x='Height', y='Weight',
        line_kws={"color": "C1"}, scatter_kws={"s": 10}, ax=ax)
ax = axs[1]
ax.set_title(f' residuals')
sns.residplot(
data=data, x='Height', y='Weight',
ax=ax)
fig.tight_layout()

## Cook's distance


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence

# fit the regression model using statsmodels library
f = 'Weight ~ Height'
model = ols(formula=f, data=data).fit()

# calculate the cooks_distance - the OLSInfluence object contains multiple influence measurements
cook_distance = OLSInfluence(model).cooks_distance
(distance, p_value) = cook_distance

## Outlier ratio

In [ ]:
# TODO complete to show the outlier count and ratio
threshold = 6/data.shape[0]
is_outlier = distance > threshold
outlier_count = ...
outlier_ratio = ...
print(f'At threshold 4/N = {threshold:0.5f}, '
      f'{outlier_count} samples are considered outliers '
      f'({outlier_ratio*100:0.2f}% of the samples)')

In [ ]:
# scatter plot - x axis (independent variable height), y-axis (dependent variable weight), size and color of the marks according to its cook's distance
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].set_title('Cook\'s distance')
sns.scatterplot(data=data, x="Height", y="Weight",
                hue=distance, size=distance, ax=axs[0],
                sizes=(50, 200), edgecolor='black', linewidth=1)


axs[1].set_title('Residual plot showing outliers')
sns.scatterplot(data=data, x="Height", y=model.resid, ax=axs[1],
                style=is_outlier, hue=is_outlier)
plt.show()
